<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#False-Discovery-Rate-&amp;-False-Omission-Rate" data-toc-modified-id="False-Discovery-Rate-&amp;-False-Omission-Rate-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>False Discovery Rate &amp; False Omission Rate</a></span></li><li><span><a href="#Calculate-by-Statsmodels" data-toc-modified-id="Calculate-by-Statsmodels-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Calculate by Statsmodels</a></span><ul class="toc-item"><li><span><a href="#Beta" data-toc-modified-id="Beta-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Beta</a></span></li><li><span><a href="#Effect-Size" data-toc-modified-id="Effect-Size-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Effect Size</a></span></li><li><span><a href="#Sample-Size" data-toc-modified-id="Sample-Size-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Sample Size</a></span></li></ul></li><li><span><a href="#Calculate-by-Simulation" data-toc-modified-id="Calculate-by-Simulation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Calculate by Simulation</a></span><ul class="toc-item"><li><span><a href="#Beta" data-toc-modified-id="Beta-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Beta</a></span></li><li><span><a href="#Raw-Effect-Size" data-toc-modified-id="Raw-Effect-Size-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Raw Effect Size</a></span></li><li><span><a href="#Sample-Size" data-toc-modified-id="Sample-Size-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Sample Size</a></span></li></ul></li></ul></div>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import IPython as ip
mpl.style.use('ggplot')
mpl.rc('font', family='Noto Sans CJK TC')
ip.display.set_matplotlib_formats('svg')

# Setup

In [3]:
# anr: actual negative rate
anr = 0.5
alpha = 0.05
beta = 0.20
# cl = 1-alpha
# power = 1-beta

# n: sample size
n_1 = 100
n_2 = 100

# sm: sample mean
# ss: sample standard deviation
sm_1 = 172
ss_1 = 5
sm_2 = 170
ss_2 = 5

# https://en.wikipedia.org/wiki/Effect_size#Cohen's_d
raw_effect_size = sm_1-sm_2
ss_pooled = np.sqrt(
    ((n_1-1)*ss_1**2 + (n_2-1)*ss_2**2) /
    (n_1+n_2-2)
)
effect_size = raw_effect_size / ss_pooled

# False Discovery Rate & False Omission Rate

$
{\displaystyle \text{false discovery rate} = \frac{ \text{observed }\alpha \cdot \text{actual negative rate} }{\text{predicted positive rate}}}
$

$
{\displaystyle \text{false omission rate} = \frac{ \text{observed }\beta \cdot \text{actual positive rate} }{\text{predicted negative rate}}}
$

In [4]:
# alpha = 0.05
# beta = 0.05
# anr = 0.94
# # FDR -> 0.4519
# # FOR -> 0.0033

# anr = 0.995
# alpha = 0.01
# beta = 0.01
# # FDR -> 0.6678
# # FOR -> 0.00005

apr = 1-anr
power = 1-beta
cl = 1-alpha

ppr = alpha*anr + power*apr
pnr = cl*anr + beta*apr

fdr = alpha*anr / ppr
for_ = beta*apr / pnr

print(fdr)
print(for_)

0.058823529411764705
0.1739130434782609


# Calculate by Statsmodels

## Beta

In [5]:
%%time
1-sm.stats.tt_ind_solve_power(
    alpha=alpha,
    effect_size=effect_size,
    nobs1=n_1,
    ratio=n_2/n_1,
    power=None,
)

CPU times: user 1.26 ms, sys: 73 µs, total: 1.33 ms
Wall time: 1.35 ms


0.19635250345692312

## Effect Size

In [6]:
%%time
sm.stats.tt_ind_solve_power(
    alpha=alpha,
    power=1-beta,
    effect_size=None,
    nobs1=n_1,
    ratio=n_2/n_1,
)*ss_pooled

CPU times: user 16.2 ms, sys: 1.45 ms, total: 17.7 ms
Wall time: 83.7 ms


1.9906955869556378

## Sample Size

In [7]:
%%time
sm.stats.tt_ind_solve_power(
    alpha=alpha,
    power=1-beta,
    effect_size=effect_size,
    ratio=1, # = n_2 / n_1
    nobs1=None,
)

CPU times: user 24.3 ms, sys: 2.45 ms, total: 26.7 ms
Wall time: 45.5 ms


99.08032683981143

# Calculate by Simulation

In [8]:
n_sim = 1000

## Beta

In [9]:
%%time
np.random.seed(20180702)
sample_1 = sp.stats.norm.rvs(loc=sm_1, scale=ss_1, size=(n_1, n_sim))
sample_2 = sp.stats.norm.rvs(loc=sm_2, scale=ss_2, size=(n_2, n_sim))
observed_beta = (sp.stats.ttest_ind(sample_1, sample_2).pvalue >= alpha).sum() / n_sim
print(observed_beta)

0.194
CPU times: user 18.1 ms, sys: 6.33 ms, total: 24.5 ms
Wall time: 32 ms


## Raw Effect Size

In [10]:
def calc_beta_given_raw_effect_size(x):
    np.random.seed(20180702)
    sample_1 = sp.stats.norm.rvs(loc=sm_1, scale=ss_1, size=(n_1, n_sim))
    # assume the sample 2 has the sample standard deviation
    sample_2 = sp.stats.norm.rvs(loc=sm_1+x, scale=ss_1, size=(n_2, n_sim))
    observed_beta = (sp.stats.ttest_ind(sample_1, sample_2).pvalue >= alpha).sum() / n_sim
    return observed_beta

In [11]:
%%time
sp.optimize.brentq(
    lambda x: calc_beta_given_raw_effect_size(x) - beta,
    3, 0
)

CPU times: user 103 ms, sys: 4.57 ms, total: 107 ms
Wall time: 113 ms


2.0355119938176798

## Sample Size

In [12]:
def calc_beta_given_sample_size(x):
    np.random.seed(20180702)
    sample_1 = sp.stats.norm.rvs(loc=sm_1, scale=ss_1, size=(int(x), n_sim))
    sample_2 = sp.stats.norm.rvs(loc=sm_2, scale=ss_2, size=(int(x), n_sim))
    observed_beta = (sp.stats.ttest_ind(sample_1, sample_2).pvalue >= alpha).sum() / n_sim
    return observed_beta

In [13]:
%%time
# === given observed_beta = beta, find the x between 200 and 100
sp.optimize.brentq(
    lambda x: calc_beta_given_sample_size(x) - beta,
    120, 80
)

CPU times: user 861 ms, sys: 30.7 ms, total: 891 ms
Wall time: 930 ms


99.99999999999855